# DSFB L-CSS HRET Correlated Group Fault Figures

Upload either a ZIP containing a single run folder or the two CSVs directly:
- group_error_comparison.csv
- group_weight_dynamics.csv

The notebook will generate two PDFs in figures-dsfb-lcss-hret/.

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
from pathlib import Path
import zipfile
import shutil

GROUP_ERROR_PATH = None
GROUP_WEIGHT_PATH = None

zip_names = [name for name in uploaded.keys() if name.lower().endswith(".zip")]
if zip_names:
    zip_name = zip_names[0]
    run_root = Path("run_data")
    if run_root.exists():
        shutil.rmtree(run_root)
    run_root.mkdir(parents=True, exist_ok=True)
    with zipfile.ZipFile(zip_name, "r") as zf:
        zf.extractall(run_root)
    group_error_candidates = list(run_root.rglob("group_error_comparison.csv"))
    group_weight_candidates = list(run_root.rglob("group_weight_dynamics.csv"))
    if group_error_candidates:
        GROUP_ERROR_PATH = group_error_candidates[0]
    if group_weight_candidates:
        GROUP_WEIGHT_PATH = group_weight_candidates[0]
else:
    group_error_candidates = list(Path(".").glob("group_error_comparison.csv"))
    group_weight_candidates = list(Path(".").glob("group_weight_dynamics.csv"))
    if group_error_candidates:
        GROUP_ERROR_PATH = group_error_candidates[0]
    if group_weight_candidates:
        GROUP_WEIGHT_PATH = group_weight_candidates[0]

print("group_error_comparison.csv:", GROUP_ERROR_PATH)
print("group_weight_dynamics.csv:", GROUP_WEIGHT_PATH)
if GROUP_ERROR_PATH is None or GROUP_WEIGHT_PATH is None:
    print("Missing one or more required CSVs.")

In [ ]:
import pandas as pd

if GROUP_ERROR_PATH is not None:
    df_error = pd.read_csv(GROUP_ERROR_PATH)
    print("group_error_comparison.csv columns:", df_error.columns.tolist())
    print(df_error.head())
else:
    df_error = None

if GROUP_WEIGHT_PATH is not None:
    df_weight = pd.read_csv(GROUP_WEIGHT_PATH)
    print("group_weight_dynamics.csv columns:", df_weight.columns.tolist())
    print(df_weight.head())
else:
    df_weight = None

In [ ]:
import matplotlib.pyplot as plt
from pathlib import Path

FIG_DIR = Path("figures-dsfb-lcss-hret")
FIG_DIR.mkdir(parents=True, exist_ok=True)

if df_error is not None:
    time_col = "time" if "time" in df_error.columns else "t"
    if "error_channel_only" in df_error.columns:
        err_a = df_error["error_channel_only"]
        err_b = df_error["error_hierarchical"]
        label_a = "channel-only"
        label_b = "hierarchical"
    else:
        err_a = df_error["err_baseline"]
        err_b = df_error["err_hierarchical"]
        label_a = "baseline"
        label_b = "hierarchical"

    fig, ax = plt.subplots()
    ax.plot(df_error[time_col], err_a, label=label_a)
    ax.plot(df_error[time_col], err_b, label=label_b)

    if "event_flag" in df_error.columns:
        mask = df_error["event_flag"] == 1
        if mask.any():
            start = df_error.loc[mask, time_col].iloc[0]
            end = df_error.loc[mask, time_col].iloc[-1]
            ax.axvspan(start, end, alpha=0.2)

    ax.set_xlabel("time")
    ax.set_ylabel("error")
    ax.legend()
    out_path = FIG_DIR / "group_error_comparison.pdf"
    fig.savefig(out_path, bbox_inches="tight")
    plt.show()
    print("Saved", out_path)
else:
    print("Skipping error plot: CSV not loaded.")

In [ ]:
import matplotlib.pyplot as plt
from pathlib import Path

FIG_DIR = Path("figures-dsfb-lcss-hret")
FIG_DIR.mkdir(parents=True, exist_ok=True)

if df_weight is not None:
    time_col = "time" if "time" in df_weight.columns else "t"
    if "group_weight" in df_weight.columns:
        w_group = df_weight["group_weight"]
        w_rep = df_weight["mean_group0_weight_hierarchical"]
        label_group = "group_weight"
        label_rep = "mean_group0_weight_hierarchical"
    else:
        w_group = df_weight["w_group"]
        w_rep = df_weight["w_channel"]
        label_group = "w_group"
        label_rep = "w_channel"

    fig, ax = plt.subplots()
    ax.plot(df_weight[time_col], w_group, label=label_group)
    ax.plot(df_weight[time_col], w_rep, label=label_rep)

    if "event_flag" in df_weight.columns:
        mask = df_weight["event_flag"] == 1
        if mask.any():
            start = df_weight.loc[mask, time_col].iloc[0]
            end = df_weight.loc[mask, time_col].iloc[-1]
            ax.axvspan(start, end, alpha=0.2)

    ax.set_xlabel("time")
    ax.set_ylabel("weight")
    ax.legend()
    out_path = FIG_DIR / "group_weight_dynamics.pdf"
    fig.savefig(out_path, bbox_inches="tight")
    plt.show()
    print("Saved", out_path)
else:
    print("Skipping weight plot: CSV not loaded.")

In [ ]:
from google.colab import files

files.download(str(Path("figures-dsfb-lcss-hret") / "group_error_comparison.pdf"))
files.download(str(Path("figures-dsfb-lcss-hret") / "group_weight_dynamics.pdf"))